In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Função para carregar as imagens
def load_images(path):
    X = []
    y = []

    for folder in os.listdir(path):
        files = os.listdir(path + folder)
        for file in tqdm(files):
            img = cv2.imread(path + folder + '/' + file)
            img = cv2.resize(img, (150, 150))
            X.append(img)
            y.append(folder)
    
    X = np.array(X)
    y = np.array(y)

    return X, y

# Carregando os dados de treinamento e teste
train_path = '../archive/seg_train/seg_train/'
test_path = '../archive/seg_test/seg_test/'
X_train, y_train = load_images(train_path)
X_test, y_test = load_images(test_path)

# Normalizando as imagens
X_train = X_train / 255.0
X_test = X_test / 255.0

# Transformando as imagens em vetores
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Codificando as labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Dividindo o conjunto de treinamento em treinamento e validação
X_train_flat, X_val_flat, y_train_encoded, y_val_encoded = train_test_split(X_train_flat, y_train_encoded, test_size=0.2, random_state=0)


100%|██████████| 510/510 [00:00<00:00, 1859.28it/s]


In [2]:
# Treinando o modelo SVM
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X_train_flat, y_train_encoded)

# Avaliando o modelo na validação
val_predictions = svm_model.predict(X_val_flat)
print("Relatório de classificação na validação:")
print(classification_report(y_val_encoded, val_predictions, target_names=label_encoder.classes_))

In [ ]:
# Avaliando o modelo no conjunto de teste
test_predictions = svm_model.predict(X_test_flat)
print("Relatório de classificação no teste:")
print(classification_report(y_test_encoded, test_predictions, target_names=label_encoder.classes_))

# Matriz de confusão
confusion_mtx = confusion_matrix(y_test_encoded, test_predictions)

plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()
